In [1]:
import numpy as np
import pickle
import rospy
import sys
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from std_srvs.srv import Empty
from nav_msgs.msg import Odometry
import time
import pandas as pd
from matplotlib import pyplot as plt
from collections import deque
import random
from keras import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.
/home/luu/Desktop/Environments/deep_ros/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/luu/Desktop/Environments/deep_ros/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/luu/Desktop/Environments/deep_ros/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home

In [2]:
cmd_vel="/cmd_vel"
laser_topic="/scan"
odom_topic="/odom"

In [17]:
class Mobile_Control:
    def __init__(self):
        self.cmd_vel = rospy.Publisher(cmd_vel,Twist,queue_size =1)
        self.subscriber = rospy.Subscriber(laser_topic,LaserScan,callback=self.laser_callback)
        self.subscriber_odo=rospy.Subscriber(odom_topic,Odometry,callback=self.odom_callback)
        self.reset = rospy.ServiceProxy("/gazebo/reset_simulation",Empty)
        self.pause = rospy.ServiceProxy("/gazebo/pause_physics",Empty)
        self.unpause = rospy.ServiceProxy("/gazebo/unpause_physics",Empty)
        self.i =0
        self.count =0
        self.vel=Twist()
        self.vel.linear.x=0
        self.vel.linear.y = 0
        self.vel.linear.z = 0
        self.vel.angular.x =0
        self.vel.angular.y = 0
        self.vel.angular.z = 0
        
    def odom_callback(self,msg1):
        self.odom_data=msg1.twist.twist
    def laser_callback(self, msg):
        self.laser_data=msg.ranges
        self.data=np.array(self.laser_data)
        self.laser_mask=np.logical_or(self.data>2, self.data<0)
        self.range_angels=np.arange(len(self.laser_data))
        self.i=0
        self.range_angels2=self.range_angels[self.laser_mask]
        for self.i in (self.range_angels2):
            self.data[self.i]=2
        self.data_laser=np.round(self.data[:])

In [18]:
rospy.init_node('Mobile_Control',anonymous=True)

In [19]:
Robot = Mobile_Control()

In [20]:
Robot.reset()

In [22]:
s=Robot.odom_data

AttributeError: 'Mobile_Control' object has no attribute 'odom_data'

In [17]:
gamma = 0.9
lr =0.7

num_episodes =15000000
epsilon =1
epsilon_decay =0.999
iters =2000
linear_actions=[2,1,0.4,0,-0.4,-1,-2]
angular_actions=[3,1.3,0,-1.3,-3]
actions=[[a1,a2] for a1 in linear_actions for a2 in angular_actions] 
#actions=[[0.4,3],[0.4,1.3],[0.4,0],[0.4,-1.3],[0.4,-3],[0.1,3],[0.1,1.3],[0.1,0],[0.1,-1.3],[0.1,-3],[0,1.3],[0,1],[0,-1.3],[-0.1,1.3],[-0.1,0],[-0.1,-1.3]]
legal_actions=len(actions)
print(actions)
print(legal_actions)
s=np.zeros([3,3,3,3,3])   
Q=np.zeros((s.size,legal_actions))
print(s.size)               

[[2, 3], [2, 1.3], [2, 0], [2, -1.3], [2, -3], [1, 3], [1, 1.3], [1, 0], [1, -1.3], [1, -3], [0.4, 3], [0.4, 1.3], [0.4, 0], [0.4, -1.3], [0.4, -3], [0, 3], [0, 1.3], [0, 0], [0, -1.3], [0, -3], [-0.4, 3], [-0.4, 1.3], [-0.4, 0], [-0.4, -1.3], [-0.4, -3], [-1, 3], [-1, 1.3], [-1, 0], [-1, -1.3], [-1, -3], [-2, 3], [-2, 1.3], [-2, 0], [-2, -1.3], [-2, -3]]
35
243


In [19]:
for p in range(num_episodes):
    
    if(abs(Robot.odom_data.linear.x)>4 or abs(Robot.odom_data.linear.y>4) or abs(Robot.odom_data.angular.z>4)):
        Robot.reset()
    s=Robot.data_laser
    #state index
    Robot.pause()
    s0=(int)(s[0]*81+s[1]*27+s[2]*9+s[3]*3+s[4])
    #epsilon greedy. to choose random actions initially when Q is all zeros
    if np.random.random()< epsilon:
        action_index=np.random.randint(0,legal_actions)
        a = actions[action_index]        
        epsilon = epsilon*epsilon_decay
        print('Took random action')
    else:
        action_index=np.argmax(Q[s0,:])
        a =actions[action_index]
        print('Took optimal action ',a)
    Robot.vel.linear.x = a[0]
    Robot.vel.angular.z=a[1]
    #print('Speed linear=%s\t angular=%s',%(a[0],a[1]) 
    Robot.unpause()
    Robot.cmd_vel.publish(Robot.vel)
    ss=Robot.data_laser
    # next state index
    Robot.pause()
    j=(int)(ss[0]*81+ss[1]*27+ss[2]*9+ss[3]*3+ss[4])
    reward=Robot.odom_data.linear.x
    print('Got reward ',reward)      
    Q[s0,action_index]= (1-lr)*Q[s0,action_index]+ lr*(reward+gamma*np.max(Q[j,:]))
    Robot.unpause()

('Took optimal action ', [2, 3])
('Got reward ', 0.9876981374028703)
('Took optimal action ', [2, 3])
('Got reward ', 0.9876981374028703)
('Took optimal action ', [2, 3])
('Got reward ', 0.9416690593414054)
Took random action
('Got reward ', 0.9416690593414054)
('Took optimal action ', [2, 3])
('Got reward ', 0.9131841267585165)
('Took optimal action ', [2, 3])
('Got reward ', 0.9131841267585165)
('Took optimal action ', [2, 3])
('Got reward ', 0.8933637271508036)
Took random action
('Got reward ', 0.8933637271508036)
('Took optimal action ', [2, 3])
('Got reward ', 0.8587528329821443)
('Took optimal action ', [2, 3])
('Got reward ', 0.8116591432883442)
('Took optimal action ', [2, 3])
('Got reward ', 0.8236427632898856)
('Took optimal action ', [2, 3])
('Got reward ', 0.8841546503229696)
('Took optimal action ', [2, 3])
('Got reward ', 0.923456566991115)
Took random action
('Got reward ', 0.9431727992215018)
Took random action
('Got reward ', 0.9628627779736568)
Took random action
('G

Took random action
('Got reward ', 0.7104904949666163)
('Took optimal action ', [1, 0])
('Got reward ', 0.7378813978171961)
('Took optimal action ', [2, 3])
('Got reward ', 0.7378813978171961)
('Took optimal action ', [2, 3])
('Got reward ', 0.7253521292793501)
Took random action
('Got reward ', 0.7253521292793501)
('Took optimal action ', [-2, -3])
('Got reward ', 0.7581430544804909)
Took random action
('Got reward ', 0.7581430544804909)
Took random action
('Got reward ', 0.723264137553298)
Took random action
('Got reward ', 0.723264137553298)
('Took optimal action ', [2, 3])
('Got reward ', 0.7012263531460159)
Took random action
('Got reward ', 0.6846661625468073)
('Took optimal action ', [0, -1.3])
('Got reward ', 0.6846661625468073)
('Took optimal action ', [0, -1.3])
('Got reward ', 0.6983792934088907)
('Took optimal action ', [0, -1.3])
('Got reward ', 0.6983792934088907)
Took random action
('Got reward ', 0.6652986041448731)
('Took optimal action ', [0, -1.3])
('Got reward ', 0.

('Got reward ', 0.029471275710598054)
('Took optimal action ', [-0.4, -3])
('Got reward ', 0.029471275710598054)
('Took optimal action ', [-0.4, -3])
('Got reward ', 0.08350355280307449)
('Took optimal action ', [-0.4, -3])
('Got reward ', 0.15293623618219293)
('Took optimal action ', [-0.4, -3])
('Got reward ', 0.15293623618219293)
('Took optimal action ', [-0.4, -3])
('Got reward ', 0.08149608767149616)
('Took optimal action ', [-0.4, -3])
('Got reward ', 0.014647535277938299)
('Took optimal action ', [-0.4, -3])
('Got reward ', 0.014647535277938299)
Took random action
('Got reward ', -0.006168196046060776)
('Took optimal action ', [-0.4, -3])
('Got reward ', -0.014211033394067634)
Took random action
('Got reward ', -0.014211033394067634)
('Took optimal action ', [-0.4, -3])
('Got reward ', 0.07230068739982165)
Took random action
('Got reward ', 0.012207626319912398)
('Took optimal action ', [-0.4, -3])
('Got reward ', 0.012207626319912398)
Took random action
('Got reward ', -0.02404

('Got reward ', -4.372618795899085e-07)
Took random action
('Got reward ', 3.064161686809098e-08)
('Took optimal action ', [1, 3])
('Got reward ', 3.064161686809098e-08)
Took random action
('Got reward ', 3.267225654259138e-07)
('Took optimal action ', [-0.4, -3])
('Got reward ', 3.267225654259138e-07)
('Took optimal action ', [2, 1.3])
('Got reward ', 6.575672801087287e-07)
('Took optimal action ', [-0.4, 1.3])
('Got reward ', 6.575672801087287e-07)
('Took optimal action ', [-2, 0])
('Got reward ', 1.2141273337038097e-06)
('Took optimal action ', [-0.4, 3])
('Got reward ', 1.2141273337038097e-06)
Took random action
('Got reward ', 0.004367427489367098)
('Took optimal action ', [-0.4, 0])
('Got reward ', 0.004367427489367098)
('Took optimal action ', [-0.4, 0])
('Got reward ', -0.0021637437261517608)
Took random action
('Got reward ', -0.01152453836031039)
('Took optimal action ', [-0.4, 0])
('Got reward ', -0.01152453836031039)
('Took optimal action ', [2, 3])
('Got reward ', -0.01152

('Got reward ', 0.03552172070500037)
('Took optimal action ', [0, 1.3])
('Got reward ', 0.02916420782658656)
('Took optimal action ', [0, 1.3])
('Got reward ', 0.02916420782658656)
Took random action
('Got reward ', 0.028420356207570514)
('Took optimal action ', [0, 1.3])
('Got reward ', 0.028420356207570514)
Took random action
('Got reward ', 0.028420356207570514)
('Took optimal action ', [0, 1.3])
('Got reward ', 0.058913446010355586)
('Took optimal action ', [0, 1.3])
('Got reward ', 0.058913446010355586)
('Took optimal action ', [0, 1.3])
('Got reward ', 0.020694726349333475)
('Took optimal action ', [0, 1.3])
('Got reward ', 0.020535191711540723)
Took random action
('Got reward ', -0.0008062045822090871)
('Took optimal action ', [0, 1.3])
('Got reward ', -0.006742685979574568)
Took random action
('Got reward ', -0.006742685979574568)
Took random action
('Got reward ', -0.006742685979574568)
('Took optimal action ', [0, 1.3])
('Got reward ', -0.0150121794079814)
('Took optimal acti

...

('Got reward ', -0.1956021411875976)
Took random action
('Got reward ', -0.1956021411875976)
('Took optimal action ', [-1, 1.3])
('Got reward ', -0.175973617618164)
Took random action
('Got reward ', -0.17974772668998562)
Took random action
('Got reward ', -0.2118834799980805)
Took random action
('Got reward ', -0.2118834799980805)
Took random action
('Got reward ', -0.2096689420663168)
Took random action
('Got reward ', -0.2096689420663168)
Took random action
('Got reward ', -0.2114108316166901)
('Took optimal action ', [1, 0])
('Got reward ', -0.16930499393593273)
Took random action
('Got reward ', -0.15196159730288114)
Took random action
('Got reward ', -0.15196159730288114)
('Took optimal action ', [-0.4, 1.3])
('Got reward ', -0.13316536416947938)
('Took optimal action ', [-0.4, 0])
('Got reward ', -0.13316536416947938)
('Took optimal action ', [0, 1.3])
('Got reward ', -0.11539338941904667)
('Took optimal action ', [1, 1.3])
('Got reward ', -0.10066161413189956)
Took random actio

('Got reward ', -0.040910431430108654)
('Took optimal action ', [0, 0])
('Got reward ', -0.08925284183007418)
Took random action
('Got reward ', -0.08925284183007418)
('Took optimal action ', [-2, 3])
('Got reward ', -0.1167336395483041)
('Took optimal action ', [-0.4, -3])
('Got reward ', -0.12706285442162146)
Took random action
('Got reward ', -0.12706285442162146)
Took random action
('Got reward ', -0.16474762977672663)
Took random action
('Got reward ', -0.17322104298601776)
Took random action
('Got reward ', -0.18867190844103804)
Took random action
('Got reward ', -0.18867190844103804)
('Took optimal action ', [0, -3])
('Got reward ', -0.15446709434176403)
Took random action
('Got reward ', -0.16843122610358782)
('Took optimal action ', [0, -3])
('Got reward ', -0.16843122610358782)
Took random action
('Got reward ', -0.19707791216208495)
('Took optimal action ', [0, -1.3])
('Got reward ', -0.19707791216208495)
Took random action
('Got reward ', -0.20678689627453536)
('Took optima

Took random action
('Got reward ', 0.23982452338325277)
Took random action
('Got reward ', 0.2536456139474108)
Took random action
('Got reward ', 0.2392252770687857)
('Took optimal action ', [2, 0])
('Got reward ', 0.2392252770687857)
Took random action
('Got reward ', 0.19655412941986394)
('Took optimal action ', [2, 0])
('Got reward ', 0.19655412941986394)
Took random action
('Got reward ', 0.2150703112670074)
('Took optimal action ', [2, 0])
('Got reward ', 0.2150703112670074)
('Took optimal action ', [2, 0])
('Got reward ', 0.23660947869359583)
Took random action
('Got reward ', 0.2608055717919864)
Took random action
('Got reward ', 0.26478859047049513)
Took random action
('Got reward ', 0.26478859047049513)
('Took optimal action ', [2, 0])
('Got reward ', 0.21417472359086412)
Took random action
('Got reward ', 0.21417472359086412)
Took random action
('Got reward ', 0.21417472359086412)
Took random action
('Got reward ', 0.19681847011615733)
('Took optimal action ', [2, 0])
('Got r

('Took optimal action ', [2, -3])
('Got reward ', -0.08972218348202432)
Took random action
('Got reward ', -0.1101246241094304)
Took random action
('Got reward ', -0.1101246241094304)
('Took optimal action ', [-0.4, 1.3])
('Got reward ', -0.11735027431249154)
Took random action
('Got reward ', -0.11735027431249154)
('Took optimal action ', [-2, 3])
('Got reward ', -0.1317157236620605)
Took random action
('Got reward ', -0.15106498144100616)
Took random action
('Got reward ', -0.15106498144100616)
('Took optimal action ', [1, -3])
('Got reward ', -0.16243304744573528)
Took random action
('Got reward ', -0.15773706793965117)
('Took optimal action ', [-2, 0])
('Got reward ', -0.15773706793965117)
Took random action
('Got reward ', -0.15773706793965117)
Took random action
('Got reward ', -0.1518283450573964)
Took random action
('Got reward ', -0.1518283450573964)
Took random action
('Got reward ', -0.1751579776087805)
('Took optimal action ', [1, 3])
('Got reward ', -0.1751579776087805)
To

KeyboardInterrupt: 

In [18]:
epsilon=0.5
num_episodes=1500000
epsilon_decay=1

In [27]:
    while 1:
        #print(Robot.data_laser)
        s=Robot.data_laser
        Robot.pause()
        s0=(int)(s[0]*81+s[1]*27+s[2]*9+s[3]*3+s[4])
        action_index=np.argmax(Q[s0,:])
        a =actions[action_index]
        Robot.vel.linear.x = a[0]
        Robot.vel.angular.z=a[1]
        Robot.unpause()
        Robot.cmd_vel.publish(Robot.vel)

KeyboardInterrupt: 

In [22]:
print(Q)

[[ 1.01721246  0.          0.         ...  0.          0.
   0.97551231]
 [ 0.08759024  0.          0.24913453 ...  0.21334838  0.
   0.24848308]
 [ 0.25058751  0.24916541  0.25072563 ...  0.25778503  0.24529285
   0.2436917 ]
 ...
 [-0.078793   -0.078793    0.         ...  0.          0.90479943
   0.        ]
 [ 2.98248964  3.10516561  2.97371708 ...  3.02887289  2.96937037
   2.9563066 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]


In [24]:
print(Q.size)

8505
